FAKE NEWS DETECTION



Attributes are as follows:

Text: Textual data in Malayalam

Label: Whether it's real or fake

In [ ]:
!pip install emoji==1.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=1f2fd83d5a2c54cf446d872c56d07a50afc05e461ce359591c54318dcf367a66
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=ec506f05df81f92a4bffdbcb754834ebe3eb3bce1bfdb14b33f9ef8724dced6c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


# **1. Preprocessiong of Data**

In [ ]:
import pandas as pd
import re
import emoji
from langdetect import detect

# Load dataset
df = pd.read_csv("Fake_train.csv")

# Preprocess text
def preprocess_text(text):
    import re
    import emoji
    from langdetect import detect

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove special characters, except emojis
    # Join the dictionary values into a string
    text = re.sub(r'[^\w\s' + ''.join(emoji.UNICODE_EMOJI['en'].keys()) + r']', '', text)

    # Replace emojis with descriptive text
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Language detection
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    return text, lang

# Apply preprocessing
df['processed_text'], df['language'] = zip(*df['text'].apply(preprocess_text))

# Filter Malayalam and English only
df.loc[df['language'].isin(['ml', 'en']), 'label'] = df.loc[df['language'].isin(['ml', 'en']), 'label'].apply(lambda x: 1 if x == "Fake" else 0)


# Map labels to binary: "Fake" -> 1, "Original" -> 0
df['label'] = df['label'].apply(lambda x: 1 if x == "Fake" else 0)


In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("Fake_train.csv")

# Preprocess text
def preprocess_text(text):
    import re
    import emoji
    from langdetect import detect

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove special characters, except emojis and join the dictionary values into a string
    text = re.sub(r'[^\w\s' + ''.join(emoji.UNICODE_EMOJI['en'].keys()) + r']', '', text)

    # Replace emojis with descriptive text
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Language detection
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    return text, lang

# Apply preprocessing
df['processed_text'], df['language'] = zip(*df['text'].apply(preprocess_text))

# Filter Malayalam and English only
df.loc[df['language'].isin(['ml', 'en']), 'label'] = df.loc[df['language'].isin(['ml', 'en']), 'label'].apply(lambda x: 1 if x == "Fake" else 0)


# Map labels to binary: "Fake" -> 1, "Original" -> 0
df['label'] = df['label'].apply(lambda x: 1 if x == "Fake" else 0)

# Display the cleaned dataset
print(df.head())


                                                text  label  \
0                  നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ      0   
1                                        Masha Allah      1   
2  അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...      0   
3    Illathentha avaru purath vidayittalland verenth      0   
4  Barana pakshathin matoru niyamam.nalla moyanth...      0   

                                      processed_text language  
0                            നലല അവതരണ സതയ പറതത വരടട       ml  
1                                        Masha Allah       so  
2  അനവഷണ കഴയമപൾ C A A യകക എതരയ കലപ പല ആകമ സവനത ആള...       ml  
3    Illathentha avaru purath vidayittalland verenth       en  
4  Barana pakshathin matoru niyamamnalla moyanth ...       tl  


# **2. Tokenization**

In [ ]:
from transformers import BertTokenizer

# Load mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the text
def tokenize_data(texts, labels, tokenizer, max_length=128):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length)
    inputs = {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
    }
    return inputs, labels

inputs, labels = tokenize_data(df['processed_text'], df['label'], tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

# **3. PyTorch Dataset and DataLoader**

In [ ]:
import torch
from torch.utils.data import Dataset

class FakeNewsDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}, torch.tensor(self.labels[idx])

# Create PyTorch Dataset
dataset = FakeNewsDataset(inputs, labels)

# Split into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16)


# **4. Model Training**

In [ ]:
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import torch.optim as optim

# Load mBERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = CrossEntropyLoss()

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader):
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs.logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 163/163 [1:02:06<00:00, 22.86s/it]


Epoch 1/3, Loss: 39.0391, Accuracy: 0.8779


100%|██████████| 163/163 [1:01:33<00:00, 22.66s/it]


Epoch 2/3, Loss: 31.8693, Accuracy: 0.8956


100%|██████████| 163/163 [1:01:23<00:00, 22.60s/it]

Epoch 3/3, Loss: 27.8872, Accuracy: 0.9152


# **5. Evaluation**

In [ ]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=['Original', 'Fake']))


              precision    recall  f1-score   support

    Original       0.94      0.94      0.94       578
        Fake       0.50      0.50      0.50        74

    accuracy                           0.89       652
   macro avg       0.72      0.72      0.72       652
weighted avg       0.89      0.89      0.89       652



# **6. Save Predictions for Test Set**

In [ ]:
# Load test data
test_df = pd.read_csv("Fake_dev.csv")  # Replace with the test dataset file path
test_df['processed_text'], _ = zip(*test_df['text'].apply(preprocess_text))

# Tokenize test data
test_inputs, _ = tokenize_data(test_df['processed_text'], None, tokenizer)

# Predict on test data
test_dataset = FakeNewsDataset(test_inputs, [0] * len(test_df))  # Dummy labels
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        inputs, _ = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}

        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())

# Save predictions
test_df['label'] = predictions
test_df[['text', 'label']].to_csv("Cognitext_task1_run.tsv", sep='\t', index=False)


In [ ]:
from google.colab import files
files.download('Cognitext_task1_run.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **7. Displaying Predictions**

In [ ]:
import pandas as pd

# Load the predictions file
predictions_df = pd.read_csv("Cognitext_task1_run.tsv", sep='\t')

# Iterate through the DataFrame and print messages with their labels
for index, row in predictions_df.iterrows():
    message = row['text']
    label = row['label']

    if label == 0:
        status = "Original"
    else:
        status = "Fake"

    print(f"Message: {message}\nStatus: {status}\n")

Message: Full.  Musilm.   Verodamum
Status: Original

Message: പക്ഷികളും മൃഗങ്ങളും ഈ ലോകത്ത് സുഖമായി ജീവിക്കുന്നു വിവരവും വിദ്യാഭ്യാസവും ഉണ്ട് എന്ന് പറയുന്ന മനുഷ്യർ മരമണ്ടൻ മാരായി ജീവിക്കുന്നു മോഹനൻ വൈദ്യരും ജേക്കബ് വടക്കഞ്ചേരി യും പറയുന്ന തലങ്ങളിലേക്ക് നിങ്ങൾ എത്താതെ തന്നെ അവരെ മണ്ടന്മാർ എന്ന് പറയുന്നതിൽ പ്രയോജനമില്ല
Status: Original

Message: ഒരു താടിക്കാരൻ പാത്രം കൊട്ടാൻ പറഞ്ഞപ്പോ ........ മറ്റൊരു ഒന്നും ഇല്ലാത്തവൻ കൈ കൊട്ടാൻ പറയുന്നു ....... 😭😭😭😭
Status: Original

Message: കുംഭളേമ<br>മറന്നോ
Status: Original

Message: ഇത് തിരുവാതിര അല്ല...... കോറോണയെ കൈകൊട്ടി കൊല്ലുകയാണ് 😂😂😂
Status: Original

Message: Kejriwal failed to control these people.
Status: Original

Message: അനുഭവിച്ചോളി അവന്മാരുടെ ഒടുക്കത്തെ കളി ഇതൊക്കെ സാദാരണ ആളാണെങ്കിൽ എപ്പോ 5000 ഫൈൻ കിട്ടിയിട്ടുണ്ടാകും ഓർമ ഇല്ലേ രണ്ടാമത്തെ ലോക്ക് ഡൗണിൽ പോലീസിന്റെ തേർവാഴ്ച
Status: Original

Message: Climax polichuu....🤣🤣
Status: Original

Message: ഈ സംഗീതത്തിന് ഈ വർഷത്തെ അവാർഡ് ഉറപ്പ്.!!😢
Status: Original

Message: ആരും അടിവര ഇടാതെ തന്ന

In [ ]:
# Fake Messages only

# Load the predictions file
predictions_df = pd.read_csv("Cognitext_task1_run.tsv", sep='\t')

# Iterate through the DataFrame and print fake messages only
for index, row in predictions_df.iterrows():
    message = row['text']
    label = row['label']

    # Check if the message is fake (label == 1)
    if label == 1:
        print(f"Message: {message}\nStatus: Fake\n")

Message: pakistane chinayude friendane avide corona udalloo
Status: Fake

Message: Indian vaidyasastram innno innalayo ullalltalla gernalista aayirakanakkinu varsam mumpumutal ullatha ketto adine niggale pole ullavanmarkku grahikkanulla kazhivillathada nammude nadinte ettavum valiya shabham pattiya kanichittu poocha ide ennuparanjal yes moolunna vidyabhasa sambhannanmar ninggal chu ducal lathi luk meen  valartham annum vaadikkunnavara
Status: Fake

Message: China ye nashipikkanam
Status: Fake

Message: Truth
Status: Fake

Message: V Good
Status: Fake

Message: june 30 vare udavum athu kazhinjale patukke kurayukayullu
Status: Fake

Message: China tana paniya itu.
Status: Fake

Message: Ithne patti onnum parayathe ipozhum china .oru response illand .ivanokke shapam kond marikkum .ethra jeevanukal aaanu myranmar karnam pokunnath .
Status: Fake

Message: Ithellam   chinayude   kali   thanne...chinayil    lakshangal    marichitundavum
Status: Fake

Message: Chinakedire oru war nammuk veenam

In [ ]:
# Original Messages

# Load the predictions file
predictions_df = pd.read_csv("Cognitext_task1_run.tsv", sep='\t')

# Iterate through the DataFrame and print fake messages only
for index, row in predictions_df.iterrows():
    message = row['text']
    label = row['label']

    # Check if the message is original (label == 0)
    if label == 0:
        print(f"Message: {message}\nStatus: Original\n")

Message: Full.  Musilm.   Verodamum
Status: Original

Message: പക്ഷികളും മൃഗങ്ങളും ഈ ലോകത്ത് സുഖമായി ജീവിക്കുന്നു വിവരവും വിദ്യാഭ്യാസവും ഉണ്ട് എന്ന് പറയുന്ന മനുഷ്യർ മരമണ്ടൻ മാരായി ജീവിക്കുന്നു മോഹനൻ വൈദ്യരും ജേക്കബ് വടക്കഞ്ചേരി യും പറയുന്ന തലങ്ങളിലേക്ക് നിങ്ങൾ എത്താതെ തന്നെ അവരെ മണ്ടന്മാർ എന്ന് പറയുന്നതിൽ പ്രയോജനമില്ല
Status: Original

Message: ഒരു താടിക്കാരൻ പാത്രം കൊട്ടാൻ പറഞ്ഞപ്പോ ........ മറ്റൊരു ഒന്നും ഇല്ലാത്തവൻ കൈ കൊട്ടാൻ പറയുന്നു ....... 😭😭😭😭
Status: Original

Message: കുംഭളേമ<br>മറന്നോ
Status: Original

Message: ഇത് തിരുവാതിര അല്ല...... കോറോണയെ കൈകൊട്ടി കൊല്ലുകയാണ് 😂😂😂
Status: Original

Message: Kejriwal failed to control these people.
Status: Original

Message: അനുഭവിച്ചോളി അവന്മാരുടെ ഒടുക്കത്തെ കളി ഇതൊക്കെ സാദാരണ ആളാണെങ്കിൽ എപ്പോ 5000 ഫൈൻ കിട്ടിയിട്ടുണ്ടാകും ഓർമ ഇല്ലേ രണ്ടാമത്തെ ലോക്ക് ഡൗണിൽ പോലീസിന്റെ തേർവാഴ്ച
Status: Original

Message: Climax polichuu....🤣🤣
Status: Original

Message: ഈ സംഗീതത്തിന് ഈ വർഷത്തെ അവാർഡ് ഉറപ്പ്.!!😢
Status: Original

Message: ആരും അടിവര ഇടാതെ തന്ന

In [ ]:
df = pd.read_csv("Cognitext_task1_run.tsv", sep='\t')
new_df = df[['text', 'label']]
new_df.columns = ['Id', 'Labels'] # Rename the columns to 'Id' and 'Labels'
new_df.to_csv("Id_Labels.tsv", sep='\t', index=False)

In [ ]:
from google.colab import files
files.download('Id_Labels.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import csv

with open('Cognitext_Malayalam.csv', 'r') as file:
    reader = csv.reader(file) # Skip the header row if present
    for row in reader:
        print(row[1])  # Print the first field of each row

Id	Labels
s
Full.  Musilm.   Verodamum	0
0
പക്ഷികളും മൃഗങ്ങളും ഈ ലോകത്ത് സുഖമായി ജീവിക്കുന്നു വിവരവും വിദ്യാഭ്യാസവും ഉണ്ട് എന്ന് പറയുന്ന മനുഷ്യർ മരമണ്ടൻ മാരായി ജീവിക്കുന്നു മോഹനൻ വൈദ്യരും ജേക്കബ് വടക്കഞ്ചേരി യും പറയുന്ന തലങ്ങളിലേക്ക് നിങ്ങൾ എത്താതെ തന്നെ അവരെ മണ്ടന്മാർ എന്ന് പറയുന്നതിൽ പ്രയോജനമില്ല	0
0
ഒരു താടിക്കാരൻ പാത്രം കൊട്ടാൻ പറഞ്ഞപ്പോ ........ മറ്റൊരു ഒന്നും ഇല്ലാത്തവൻ കൈ കൊട്ടാൻ പറയുന്നു ....... 😭😭😭😭	0
0
കുംഭളേമ<br>മറന്നോ	0
0
ഇത് തിരുവാതിര അല്ല...... കോറോണയെ കൈകൊട്ടി കൊല്ലുകയാണ് 😂😂😂	0
0
Kejriwal failed to control these people.	0
0
അനുഭവിച്ചോളി അവന്മാരുടെ ഒടുക്കത്തെ കളി ഇതൊക്കെ സാദാരണ ആളാണെങ്കിൽ എപ്പോ 5000 ഫൈൻ കിട്ടിയിട്ടുണ്ടാകും ഓർമ ഇല്ലേ രണ്ടാമത്തെ ലോക്ക് ഡൗണിൽ പോലീസിന്റെ തേർവാഴ്ച	0
0
Climax polichuu....🤣🤣	0
0
ഈ സംഗീതത്തിന് ഈ വർഷത്തെ അവാർഡ് ഉറപ്പ്.!!😢	0
0


In [ ]:
import csv

# Function to split each string in the row into 'Id' and 'Label'
def split_string(text):
    # Split the string into words
    parts = text.split()

    if len(parts) > 1:
        # The last word is the label, and the rest is the Id
        label = parts[-1]
        id_text = ' '.join(parts[:-1])  # Join all parts except the last one
        return id_text, label
    else:
        # If there is only one word, the entire text is considered as 'Id' and label is '0'
        return text, '0'

# Open the input CSV file for reading
with open('Cognitext_Malayalam.csv', mode='r', newline='', encoding='utf-8') as infile:
    # Create a CSV reader
    csvreader = csv.reader(infile)

    # Open the output CSV file for writing
    with open('new_file.csv', mode='w', newline='', encoding='utf-8') as outfile:
        # Create a CSV writer
        csvwriter = csv.writer(outfile)

        # Write the header row
        csvwriter.writerow(['Id', 'Labels'])

        # Skip the header if the input CSV has one
        next(csvreader)

        # Process each row in the input CSV
        for row in csvreader:
            text_label = row[0]  # The string is in the first column (index 0)
            id_text, label = split_string(text_label)

            # Write the processed row to the output CSV
            csvwriter.writerow([id_text, label])

print("New CSV file has been created successfully!")


New CSV file has been created successfully!


In [ ]:
with open('new_file.csv', 'r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):  # Use enumerate to get row index
        if i == 15:  # Check if it's the 13th row (index 12)
            print(row)  # Print the entire row
            break  # Exit the loop after printing

['Andha', 'vishvasavum']


In [ ]:
with open('new_file.csv', 'r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):  # Use enumerate to get row index
        if i == 13:  # Check if it's the 13th row (index 12)
            print(row)  # Print the entire row
            break  # Exit the loop after printing

['Kandagasheni &quot;SHAAPAM&quot; kondepovu .....', '0']


In [ ]:
from google.colab import files
files.download('new_FILE.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import csv

# Function to check and modify the Label if it's not 0 or 1
def modify_label(label):
    if label not in ['0', '1']:  # If the label is not 0 or 1
        return '0'  # Replace with 0
    return label  # If it's already 0 or 1, keep it unchanged

# Open the input CSV file for reading
with open('Cognitext_Malayalam.csv', mode='r', newline='', encoding='utf-8') as infile:
    # Create a CSV reader
    csvreader = csv.reader(infile)

    # Open the output CSV file for writing
    with open('new_FILE.csv', mode='w', newline='', encoding='utf-8') as outfile:
        # Create a CSV writer
        csvwriter = csv.writer(outfile)

        # Read the header from the input file
        header = next(csvreader)  # Read header row

        # Write the header to the output file (with 'Id' and 'Labels' as the header)
        csvwriter.writerow(['Id', 'Labels'])

        # Process each row in the input CSV
        for row in csvreader:
            if row:  # Ensure the row is not empty
                # Split the row into words
                parts = row[0].split()

                if len(parts) > 1:
                    # The last word is the label, and the rest is the Id
                    label = parts[-1]
                    id_text = ' '.join(parts[:-1])  # Join all parts except the last one

                    # Modify the label if it's not 0 or 1
                    label = modify_label(label)

                    # Write the modified row to the output CSV
                    csvwriter.writerow([id_text, label])
                else:
                    # If there's only one word, treat it as the 'Id' and the label will be '0'
                    csvwriter.writerow([parts[0], '0'])

print("New CSV file has been created successfully!")



New CSV file has been created successfully!


In [ ]:
with open('new_FILE.csv', 'r') as file:
    reader = csv.reader(file) # Skip the header row if present
    for row in reader:
        print(row[1])  # Print the first field of each row

Labels
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Fake
Original
Original
Fake
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Fake
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Fake
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Fake
Original
Original
Original
Original
Original
Fake
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Fake
Original
Original
Orig

In [ ]:
import csv

# Function to replace label values with 'Fake' and 'Original'
def modify_label(label):
    if label == '1':  # If label is 1, replace with 'Fake'
        return 'Fake'
    elif label == '0':  # If label is 0, replace with 'Original'
        return 'original'
    return label  # If the label is something else, leave it unchanged

# Open the CSV file for reading
with open('new_FILE.csv', mode='r', newline='', encoding='utf-8') as infile:
    # Create a CSV reader
    csvreader = csv.reader(infile)

    # Read all rows from the file
    rows = list(csvreader)

# Open the same file for writing (overwrite mode)
with open('new_FILE.csv', mode='w', newline='', encoding='utf-8') as outfile:
    # Create a CSV writer
    csvwriter = csv.writer(outfile)

    # Write the header to the output file
    header = rows[0]
    csvwriter.writerow(header)  # Write the header row as is

    # Process each row in the CSV
    for row in rows[1:]:  # Skip the header row
        id_text = row[0]  # The Id value
        label = row[1]    # The label value

        # Modify the label value if necessary
        new_label = modify_label(label)

        # Write the modified row back to the file
        csvwriter.writerow([id_text, new_label])

print("Labels have been updated successfully in the file!")


Labels have been updated successfully in the file!


In [ ]:
import csv

# Open the CSV file for reading
with open('new_FILE.csv', mode='r', newline='', encoding='utf-8') as infile:
    # Create a CSV reader
    csvreader = csv.reader(infile)

    # Use the built-in `sum` function to count rows, excluding the header
    row_count = sum(1 for row in csvreader)

print(f'Total number of rows: {row_count}')


Total number of rows: 816


In [ ]:
import csv

# Open the CSV file for reading
with open('Fake_dev.csv', mode='r', newline='', encoding='utf-8') as infile:
    # Create a CSV reader
    csvreader = csv.reader(infile)

    # Use the built-in `sum` function to count rows, excluding the header
    row_count = sum(1 for row in csvreader)

print(f'Total number of rows: {row_count}')


Total number of rows: 816
